In [ ]:
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
from matplotlib.figure import Figure
import pandas as pd
from scipy.stats import linregress
import requests, json, datetime, sys, os
import tkinter as tk

def dataFigure(date_time, value, date_time_seconds, time_frame, key): #Function
    plt.clf() #Clear current figure.
    plt.cla() #Clear current Axis
    plt.xticks(rotation = 30) #Rotates X-Axis 
    plt.grid() #Sets Grid
    plt.tight_layout(rect=[0.05, 0.05, 0.95, 0.95]) #Fits figure cleanly
    plt.xlabel('Date') #Sets X-Label
    plt.ylabel('Price per Coin') #Sets Y-Label
    plt.plot(date_time, value, color = 'black') #Plots information
    plt.title(f"{key.capitalize()} price over {time_frame} days") #Sets Title
    regression = [] #Empty list
    (slope, intercept, rvalue, pvalue, stderr) = linregress(date_time_seconds, value)
    #Using LineRegress import
    regress_values = [] #Empty list
    for i in date_time_seconds: #For loop utilizing 
        regress_values.append(i * slope + intercept) #Appends Regress_Values with parameters output
    plt.plot(date_time, regress_values,'-.', color = 'Blue') #Plots LineRegression
    try: #Attempt
        os.remove(f'{key.capitalize()}.png') #Removes png file for saving. Use to reduce clutter
        print('Output File already exists. Deleting...')  
        print(f'Creating new file for {key.capitalize()} as png')
        plt.savefig(f'{key.capitalize()}.png') #Save new figure
    except OSError:
        print(f'Creating save file for {key.capitalize()} as png')
        plt.savefig(f'{key.capitalize()}.png') #Iff no figure exists, saves figure to file
        pass 
    

def checkCoinPrice(url, key): #Function
    response = requests.get(url) #Catch information 
    data_construct = response.json() #Read information
    if data_construct == {"error":"Could not find coin with the given id"} or data_construct == {'error': 'invalid vs_currency'}:
        print("There was an issue with variables. Check work and try again.") #Above checks for 2 possibilities
    date_time = [] #Empty list
    
    value = [] #Empty list
    date_time_seconds = [] #Empty list
    for row in data_construct['prices']: # For loop
        date_time.append(datetime.datetime.utcfromtimestamp(int(row[0]/1000)))
        ##Appends list of UNIX EPOCH timestamp. Divide by 1000 to change to seconds.
        value.append(row[1]) #Appends list
        date_time_seconds.append(int((row[0]/1000))) #Appends list #BASELINE ORIGINAL
        
        
    date = [] #Empty list
    time = [] #Empty list
    for convertline in date_time: #For loop
        split = str(convertline).split(' ') #Breaks date and time at ' ' (Space)
        date.append(split[0]) #Append list
        time.append(split[1]) #Append list
    data_frame = pd.DataFrame(date, time) #DataFrame creation
    data_frame = data_frame.reset_index().rename(columns= {'index': 'Time', 0: 'Date'}) #Resets Index, Renames Columns
    data_frame[key] = value #Creates new list based on Key within DataFrame
    return(date_time, value, date_time_seconds, data_frame) #Returns information
    
def process(coinIndex, days):
    if not days.isnumeric(): #Checks to see if Numeric
        print("Days needs to be a number!")
        return
    quick_keys = {1: 'bitcoin', 2: 'litecoin', 3: 'ethereum', 4: 'ripple'} #Dictionary
    time_frame = int(days) #Input Integer
    listOfDicts = None #Initialized
    key = quick_keys[coinIndex] #Set variable to correct key/value 
    url = f'https://api.coingecko.com/api/v3/coins/{key}/market_chart?vs_currency=usd&days={time_frame}'
    date_time, value, date_time_seconds, data_frame = checkCoinPrice(url, key) #Runs function
        
        
    if listOfDicts is None: #Checks for value
        listOfDicts = data_frame #changes to value
    else: 
        listOfDicts[key] = data_frame[key] #Changes value to key
        
    dataFigure(date_time, value, date_time_seconds, time_frame, key) #Runs Function
    print(listOfDicts) 

window = tk.Tk() #Uses TK library
userOptions = tk.LabelFrame(window, text="Options") #Sets Title to list
userOptions.pack(fill="x", anchor="nw") #sends to GUI

radioButtonFrame = tk.Frame(userOptions) #Groups widgets
radioButtonFrame.pack(anchor="w") #Sends to GUI
coinVar = tk.IntVar() 
#Each Radiobutton is a clickable link for the GUI
bitcoin = tk.Radiobutton(radioButtonFrame, text="Bitcoin", variable=coinVar, value=1)
bitcoin.pack(anchor="w") #Sends to GUI
ethereum = tk.Radiobutton(radioButtonFrame, text="Ethereum", variable=coinVar, value=2)
ethereum.pack(anchor="w") #Sends to GUI
litecoin = tk.Radiobutton(radioButtonFrame, text="Litecoin", variable=coinVar, value=3)
litecoin.pack(anchor="w") #Sends to GUI
ripple = tk.Radiobutton(radioButtonFrame, text="Ripple", variable=coinVar, value=4)
ripple.pack(anchor="w") #Sends to GUI

daysFrame = tk.Frame(userOptions)
daysFrame.pack(anchor="w") #Sends to GUI
daysLabel = tk.Label(daysFrame, text="Number of Days: ") #Insert string 
daysLabel.pack(anchor="w", side="left") #Sends to GUI
days = tk.StringVar()
daysInput = tk.Entry(daysFrame, textvariable=days) #Text Input
daysInput.pack(anchor="w", side="left") #Sends to GUI

processButton = tk.Button(userOptions, text="Process", command= lambda: process(coinVar.get(), days.get()))
processButton.pack(anchor="w")
#Runs data input with clicked RadioFrame
plotFrame = tk.LabelFrame(window, text="Plot") #Sets title to Plot
plotFrame.pack(fill="x", anchor="nw") #Sends to GUI

f = plt.figure(figsize=(12,6))  #Figure size

canvas = FigureCanvasTkAgg(f, plotFrame) 
canvas.draw() #Puts information onto figure
canvas.get_tk_widget().pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

toolbar=NavigationToolbar2Tk(canvas, plotFrame)
toolbar.update() #Allows you to utilize things to zoom in
canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
window.mainloop() #Allows for clicks